# Assignment 2 
## Samridhi Gupta  a1901641

## A. Loading dataset

### 1. All Imports

In [2]:
import json 
import string
import nltk
import re
import pandas as pd
import seaborn as sns
#nltk.download('stopwords')
from nltk.corpus import stopwords 
#nltk.download('punkt')
from nltk.tokenize import word_tokenize 
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import matplotlib.pyplot as plt
from collections import Counter
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud

#from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

### 2. Reading Data

In [3]:
# Load the dataset
news_data = pd.read_csv("news_dataset.csv", encoding = 'latin-1')

# Display the first few rows of the dataset
print(news_data.head())
news_data.shape[0]

      id               author        date  year month         topic  \
0  17307       Marlise Simons   1/01/2017  2017     1  architecture   
1  17292          Andy Newman  31/12/2016  2016    12           art   
2  17298  Emma G. Fitzsimmons   2/01/2017  2017     1      business   
3  17311           Carl Hulse   3/01/2017  2017     1      business   
4  17339        Jim Rutenberg   5/01/2017  2017     1      business   

                                             article  
0  PARIS  ?   When the Islamic State was about to...  
1  Angels are everywhere in the Mu?iz family?s ap...  
2  Finally. The Second Avenue subway opened in Ne...  
3  WASHINGTON  ?   It?s   or   time for Republica...  
4  For Megyn Kelly, the shift from Fox News to NB...  


1000

### 3. Removing missing and duplicate rows

In [4]:
# removing rows with no information
news_data.dropna(inplace=True)
# Display the cleaned DataFrame
print(news_data.head())
news_data.shape[0]  # outputs 994, 6 rows removed

      id               author        date  year month         topic  \
0  17307       Marlise Simons   1/01/2017  2017     1  architecture   
1  17292          Andy Newman  31/12/2016  2016    12           art   
2  17298  Emma G. Fitzsimmons   2/01/2017  2017     1      business   
3  17311           Carl Hulse   3/01/2017  2017     1      business   
4  17339        Jim Rutenberg   5/01/2017  2017     1      business   

                                             article  
0  PARIS  ?   When the Islamic State was about to...  
1  Angels are everywhere in the Mu?iz family?s ap...  
2  Finally. The Second Avenue subway opened in Ne...  
3  WASHINGTON  ?   It?s   or   time for Republica...  
4  For Megyn Kelly, the shift from Fox News to NB...  


994

## B. User input

The system first prompts the user to input an article ID and a question. Then, it calls the get_article_by_id function to retrieve the text of the article corresponding to the provided ID from the news_data DataFrame. If the article ID exists in the DataFrame, the function returns the article text; otherwise, it notifies the user that the article ID was not found

In [5]:
# getting the article corresponding to the ID enetered by the user
def get_article_by_id(news_data, article_id):
    # Check if the article ID exists in the dataframe
    if article_id in news_data['id'].values:
        # Retrieve the article text corresponding to the given ID
        article_text = news_data.loc[news_data['id'] == article_id, 'article'].values[0]
        return article_text
    else:
        return "Article ID not found in the data"

# Prompt the user to enter an ID
article_id = input("Enter the ID of the article: ")
question = input("Please enter your question: ")

# Call the function to get the article text
article_text = get_article_by_id(news_data, int(article_id))

Enter the ID of the article: 17308
Please enter your question: What is the famous song by George Michael?


In [6]:
#print(article_text)

## C. Preprocessing

-  Lemmatization, a text pre-processing technique in NLP, reduces words to their root form, aiding in identifying similarities and improving the accuracy of analyses (Built In 2013). 
-  It is vital in information retrieval as it standardizes words to their base form, enhancing the matching process between queries and documents. 
-  By reducing words to their root meaning, lemmatization improves the precision and recall of retrieval systems, ensuring more accurate and comprehensive results. 
-  Stop word removal is critical in information retrieval systems as it helps filter out common words that carry little semantic meaning, thereby reducing noise and improving the relevance of search results. 
-  Initially, I had decided to remove stop-words but as I experimented more, I was afraid that removing stop-words would remove important pronouns. Hence, I lemmatized the article chosen by the user, and removed all kinds of punctuation except full-stops. I didn’t remove full stops, because that would help the model recognize the end of a sentence. 

In [7]:
# splitting an article into sentences
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(article_text)

In [8]:
sents = doc.sents
sents = [i.text for i in sents]
#print(sents)

In [9]:
print(sents)

['Pop music and fashion never met cuter than in George Michael?s ?', 'Freedom! ?', '90? video.', 'Sure, other catwalk favorites starred in music videos during that era, and fashion photographers like Steven Meisel and Herb Ritts were recruited by stars such as Madonna, Chris Isaak and Janet Jackson to burnish their visuals.', 'But the convergence of a   maker?s irresistible rhythms and lyrics, a group of models at the peak of their fame and a director on the verge of his own runaway success gave ?', 'Freedom! ?', '90?', 'a jolt of stylish energy that has yet to fade after more than a  .', 'Directed by David Fincher, who was at the start of his film career, and filmed over several days at Merton Park Studios in London, the video from Mr. Michael?s 1990 album, ?', 'Listen Without Prejudice Vol. 1,?', 'brought together five models  ?', '  Naomi Campbell, Linda Evangelista, Christy Turlington, Cindy Crawford and Tatjana Patitz  ?', '  who were then the queens of fashion.', 'It runs for six

In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

def preprocess_sentence(sentence):
#     stop_words = set(stopwords.words('english'))
    stop_words = set()
    lemmatizer = WordNetLemmatizer()
    word_tokens = nltk.word_tokenize(sentence)
    
    # Remove punctuation marks except full stops
    punctuation_filtered = [word for word in word_tokens if word not in string.punctuation or word == '.']
    
    # Remove stopwords and lemmatize words
    filtered_sentence = [lemmatizer.lemmatize(word.lower()) for word in punctuation_filtered if word.lower() not in stop_words]
    
    # Join the words back into a sentence
    return ' '.join(filtered_sentence)

preprocessed_sentences = [preprocess_sentence(sents) for sents in sents]
print(preprocessed_sentences)

['pop music and fashion never met cuter than in george michael s', 'freedom', '90 video .', 'sure other catwalk favorite starred in music video during that era and fashion photographer like steven meisel and herb ritts were recruited by star such a madonna chris isaak and janet jackson to burnish their visuals .', 'but the convergence of a maker s irresistible rhythm and lyric a group of model at the peak of their fame and a director on the verge of his own runaway success gave', 'freedom', '90', 'a jolt of stylish energy that ha yet to fade after more than a .', 'directed by david fincher who wa at the start of his film career and filmed over several day at merton park studio in london the video from mr. michael s 1990 album', 'listen without prejudice vol . 1', 'brought together five model', 'naomi campbell linda evangelista christy turlington cindy crawford and tatjana patitz', 'who were then the queen of fashion .', 'it run for six and a half minute and it production also gathered 

## D. Coreference Resolution Utility

The model takes the article, and pre-processes the context. It is then split into sentences. Coreference resolution involves identifying all instances in a text where different expressions refer to the same entity. This task is essential for various advanced NLP applications like document summarization, question answering, and information extraction, as it enhances natural language understanding by resolving ambiguous references (The Stanford NLP Group n.d.).

Coreference occurs when a subsequent mention in a text refers back to an entity or event introduced earlier. In essence, two expressions are said to corefer when they denote the same entity or event. In a coreference resolution system, the process involves identifying mentions of entities and subsequently grouping mentions that refer to the same entity. Each mention is represented by a continuous sequence of words. These systems learn from annotated data where mentions are grouped into entity clusters, and their performance is evaluated by comparing the system's grouping of mentions to the predefined clusters in the test dataset (Explosion.ai 2022). Coreference resolution serves as a highly adaptable tool applicable across a spectrum of NLP tasks, including text understanding, information extraction, machine translation, sentiment analysis, and document summarization (NeuroSys n.d.). 

In [11]:
#!pip install spacy-experimental

In [12]:
#!pip install https://github.com/explosion/spacy-experimental/releases/download/v0.6.0/en_coreference_web_trf-3.4.0a0-py3-none-any.whl#egg=en_coreference_web_trf

In [13]:
#!pip install spacy-transformers

In [14]:
# loading the coreference resolution pipeline of spacy
nlp = spacy.load("en_coreference_web_trf")

-  The model makes use of coreference resolution with the spaCy libraries. The code loads a pre-trained coreference resolution model from the spaCy library named "en_coreference_web_trf."
-  This model is specifically trained for English text and utilizes transformer-based architecture, likely incorporating techniques from the Transformer model architecture.

In [15]:
text = ''.join(preprocessed_sentences)
#print(text)

-  Once loaded, this model can be used for resolving coreferences in text data. The library makes use of the pre-processed sentences to form clusters. 
-  The result is comprised of coreference clusters extracted from the article (context). Each cluster represents mentions of the same entity or concept within the text. 
-  For example, cluster 'coref_clusters_1' includes mentions related to "fashion photographer like Steven Meisel and Herb Ritts," while 'coref_clusters_2' contains mentions associated with "a director on the verge of his own runaway success" and "David Fincher who was at the start of his film career." 
-  These clusters help in understanding how different expressions within the text refer to the same entity or concept, aiding in tasks like text understanding and information extraction. 

In [16]:
# Process each sentence and resolve coreferences
doc = nlp(text)
doc.spans

{'coref_clusters_1': [fashion photographer like steven meisel and herb ritts were, their visuals], 'coref_clusters_2': [a director on the verge of his own runaway success gavefreedom90a, a director on the verge of his own, david fincher who wa at the start of his film career and, his film], 'coref_clusters_3': [george michael sfreedom90 video .sure, the video from, .it run, it production, the video s most, the video .what, the video i, the finale and, the video would], 'coref_clusters_4': [george michael sfreedom90, michael s 1990, michael s unexpected, george in, he saidi, i need, you d, u in, him designing, george wa, london .david fincher knew, he wanted, .he didn, he knew, the song he, he wanted, george a, him .when, he sang, he wa, him with, he invited, his home, he had, david fincher had, george .tatjana, .george michael wasn, him a, him for, .george wa, .he wa, him he, he wa, big george michael fan, him all, .george wa, the producer said, we shoot, he saiddo], 'coref_clusters_5'

-  The next code snippet processes coreference clusters extracted from a document using the spaCy library. 
-  It first creates a dictionary named main_nouns to store the main noun for each coreference cluster. 
-  Then, it iterates through the document tokens, identifies pronouns with associated coreference clusters, and replaces the pronouns with the corresponding main noun from the main_nouns dictionary. 
-  Finally, it reconstructs the document text with the pronouns replaced by the main nouns and prints the updated article text.

In [17]:
# dictionary to store main nouns for each coreference cluster
coreference_clusters = doc.spans
main_nouns = {}

for cluster_key, cluster_values in coreference_clusters.items():
    # Extract the main noun from each cluster
    main_noun = cluster_values[0].text
    # Store the main noun for the cluster
    main_nouns[cluster_key] = main_noun

# Replace pronouns with corresponding nouns
for token in doc:
    if token.pos_ == "PRON" and token._.coref_clusters:
        # Find the cluster key for the pronoun
        cluster_key = token._.coref_clusters[0].id
        # Replace the pronoun with the corresponding noun
        token.text = main_nouns[cluster_key]

new_article_text = " ".join(token.text_with_ws for token in doc)
print(new_article_text)

pop  music  and  fashion  never  met  cuter  than  in  george  michael  sfreedom90  video  .sure  other  catwalk  favorite  starred  in  music  video  during  that  era  and  fashion  photographer  like  steven  meisel  and  herb  ritts  were  recruited  by  star  such  a  madonna  chris  isaak  and  janet  jackson  to  burnish  their  visuals  .but  the  convergence  of  a  maker  s  irresistible  rhythm  and  lyric  a  group  of  model  at  the  peak  of  their  fame  and  a  director  on  the  verge  of  his  own  runaway  success  gavefreedom90a  jolt  of  stylish  energy  that  ha  yet  to  fade  after  more  than  a  .directed  by  david  fincher  who  wa  at  the  start  of  his  film  career  and  filmed  over  several  day  at  merton  park  studio  in  london  the  video  from  mr .  michael  s  1990  albumlisten  without  prejudice  vol  .  1brought  together  five  modelnaomi  campbell  linda  evangelista  christy  turlington  cindy  crawford  and  tatjana  patitzwho  were 

## E. Vectorization

TF-IDF is a prevalent statistical technique employed in natural language processing and information retrieval, gauging the significance of a term within a document relative to a corpus. Through text vectorization, words in a document are converted into numerical importance values, with TF-IDF serving as a prominent scoring scheme among various vectorization methods. 

-  The next part of the code utilizes the TfidfVectorizer from scikit-learn to compute TF-IDF scores for preprocessed sentences. 
-  Firstly, the TfidfVectorizer is initialized. Then, the preprocessed sentences are fitted and transformed using the vectorizer. 
-  Finally, the TF-IDF matrix and the corresponding feature names are printed. It's essential to ensure that the 'preprocessed_sentences' variable contains a list of preprocessed sentences before executing this code. 
-  Additionally, the output will display the TF-IDF matrix and the feature names obtained from the vectorizer. 
-  In the context of TF-IDF vectorization, feature names refer to the individual terms or words present in the corpus that are used as features in the vectorized representation of the documents. 
-  These feature names are typically obtained from the vocabulary created during the vectorization process. 
-  Each feature name corresponds to a unique term in the corpus, and they are used to represent the columns in the TF-IDF matrix. 
-  Feature names allow for the identification of the important terms in the corpus based on the TF-IDF scores. 

In [18]:
# Performing tf-idf on preprocessed sentences
# initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# fit and transform the preprocessed sentences
tfidf_sents = tfidf_vectorizer.fit_transform(preprocessed_sentences)

# get the feature names
feature_names = tfidf_vectorizer.get_feature_names_out()

print("TF-IDF Matrix:")
print(tfidf_sents.toarray())

print("\nFeature Names:")
print(feature_names)

TF-IDF Matrix:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

Feature Names:
['000' '10' '11' '15' '1990' '37' '40' '50' '53' '76' '80' '90' 'about'
 'active' 'actress' 'actually' 'affect' 'after' 'afternoon' 'agent' 'aid'
 'aired' 'alan' 'album' 'all' 'almost' 'also' 'although' 'always'
 'amazing' 'america' 'an' 'and' 'another' 'any' 'anymore' 'anything'
 'ardent' 'are' 'area' 'around' 'arrived' 'artist' 'at' 'audrey' 'ava'
 'away' 'back' 'bad' 'bag' 'bar' 'basically' 'be' 'beautiful' 'beauty'
 'because' 'become' 'been' 'before' 'being' 'best' 'big' 'bit' 'black'
 'blond' 'body' 'bolshie' 'booked' 'boy' 'briefly' 'brilliant' 'bring'
 'british' 'brought' 'brown' 'burnish' 'business' 'bustier' 'but' 'by'
 'called' 'calling' 'came' 'camilla' 'campbell' 'can' 'candis' 'capstone'
 'care' 'career' 'carol' 'catwalk' 'cayne' 'chaise' 'changed' 'character'
 'cheap' 'child' 'choir' 'cho

## F. Cosine similarity 

Cosine similarity quantifies the similarity between two vectors in an inner product space by calculating the cosine of the angle between them. It assesses whether the vectors are aligned in a similar direction, indicating similarity. In text analysis, cosine similarity is frequently employed to gauge document similarity, enabling comparisons between documents based on their content (Science Direct 2012). 

-  The model calculates the cosine similarity between the user’s question and the set of preprocessed sentences using TF-IDF vectorization. 
-  Firstly, the question and preprocessed sentences are combined into a list named 'documents'. Then, TF-IDF vectorization is applied using scikit-learn's TfidfVectorizer to convert the text data into numerical vectors. 
-  Next, cosine similarity is computed between the question vector (first row of the TF-IDF matrix) and each sentence vector (remaining rows of the TF-IDF matrix). 
-  Finally, the cosine similarity scores between the question and each sentence are printed. 
-  This approach allows for the comparison of the question with each sentence based on their content similarity, providing insights into the relevance of each sentence to the question.

In [19]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine the question and sentences for vectorization
documents = [question] + preprocessed_sentences

# Vectorize the documents using TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# Compute cosine similarity between the question and each sentence
cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])

# Print cosine similarities
for i, sentence in enumerate(preprocessed_sentences):
    print(f"Cosine similarity between question and sentence {i + 1}: {cosine_similarities[0][i]}")

Cosine similarity between question and sentence 1: 0.16320696713390243
Cosine similarity between question and sentence 2: 0.0
Cosine similarity between question and sentence 3: 0.0
Cosine similarity between question and sentence 4: 0.05323882024519466
Cosine similarity between question and sentence 5: 0.042435663762425205
Cosine similarity between question and sentence 6: 0.0
Cosine similarity between question and sentence 7: 0.0
Cosine similarity between question and sentence 8: 0.0
Cosine similarity between question and sentence 9: 0.1391578644920517
Cosine similarity between question and sentence 10: 0.0
Cosine similarity between question and sentence 11: 0.0
Cosine similarity between question and sentence 12: 0.0
Cosine similarity between question and sentence 13: 0.03585932777455805
Cosine similarity between question and sentence 14: 0.045277773468196315
Cosine similarity between question and sentence 15: 0.0
Cosine similarity between question and sentence 16: 0.0
Cosine similarit

After calculating the cosine similarities of every sentence, the model extracts the indexes of sentences with cosine similarity scores exceeding a threshold of 0.1, 0.2, and 0.3 respectively, and sorts them based on their similarity scores. Then, it prints the top similar sentences along with their corresponding similarity scores. By setting a similarity threshold and retrieving the most similar sentences, this approach facilitates the identification of sentences closely related to the question, aiding in information retrieval.

In [20]:
# selecting scores >0.1
# print top 5 similarity indexes where similarity score is greater than 0.1
top_indexes = []
for i, similarity_score in enumerate(cosine_similarities[0]):
    if similarity_score > 0.1:
        top_indexes.append(i)

# Sort the indexes based on similarity score
top_indexes.sort(key=lambda x: cosine_similarities[0][x], reverse=True)

# Print top 5 similarity indexes
print("Top 5 similar sentences:")
for idx in top_indexes[:5]:
    print(f"Sentence {idx + 1}: {preprocessed_sentences[idx]} (Similarity Score: {cosine_similarities[0][idx]})")

Top 5 similar sentences:
Sentence 159: and i turned on the radio and on came george michael s new song . (Similarity Score: 0.2774606557034096)
Sentence 107: i wa a big george michael fan . (Similarity Score: 0.26286059581552235)
Sentence 181: there s water dripping down the wall and george michael s leather jacket is on fire and the model are naked . (Similarity Score: 0.25130061081726784)
Sentence 66: what i remember most is the day being very long and at the end of the day the red wine would come out . (Similarity Score: 0.21691289087616)
Sentence 162: it wa his song . (Similarity Score: 0.21164102337426283)


In [21]:
# selecting scores >0.2
# print top 5 similarity indexes where similarity score is greater than 0.2
top_indexes = []
for i, similarity_score in enumerate(cosine_similarities[0]):
    if similarity_score > 0.2:
        top_indexes.append(i)

# Sort the indexes based on similarity score
top_indexes.sort(key=lambda x: cosine_similarities[0][x], reverse=True)

# Print top 5 similarity indexes
print("Top 5 similar sentences:")
for idx in top_indexes[:5]:
    print(f"Sentence {idx + 1}: {preprocessed_sentences[idx]} (Similarity Score: {cosine_similarities[0][idx]})")

Top 5 similar sentences:
Sentence 159: and i turned on the radio and on came george michael s new song . (Similarity Score: 0.2774606557034096)
Sentence 107: i wa a big george michael fan . (Similarity Score: 0.26286059581552235)
Sentence 181: there s water dripping down the wall and george michael s leather jacket is on fire and the model are naked . (Similarity Score: 0.25130061081726784)
Sentence 66: what i remember most is the day being very long and at the end of the day the red wine would come out . (Similarity Score: 0.21691289087616)
Sentence 162: it wa his song . (Similarity Score: 0.21164102337426283)


In [22]:
# selecting scores >0.3
# print top 5 similarity indexes where similarity score is greater than 0.3
top_indexes = []
for i, similarity_score in enumerate(cosine_similarities[0]):
    if similarity_score > 0.3:
        top_indexes.append(i)

# Sort the indexes based on similarity score
top_indexes.sort(key=lambda x: cosine_similarities[0][x], reverse=True)

# Print top 5 similarity indexes
print("Top 5 similar sentences:")
for idx in top_indexes[:5]:
    print(f"Sentence {idx + 1}: {preprocessed_sentences[idx]} (Similarity Score: {cosine_similarities[0][idx]})")

Top 5 similar sentences:


## G. Testing Utility 

The test utility sets up a test scenario for evaluating a question-answering system. It provides a list of 10 questions (test_questions) and corresponding answers (test_answers). Each question is paired with a sentence containing the answer to that question. This setup allows for testing the question-answering system by comparing the answers generated by the system against the provided ground truth answers. By evaluating the system's performance against known correct answers, one can assess its accuracy and effectiveness in answering questions. 

Furthermore, I defined a function match_question_to_sentence to identify sentences containing answers to user questions based on predefined rules. The rules specify patterns to match against sentences. When a user question is provided, the function searches for matches between the question and sentences using regular expressions and the defined rules. For instance, if the user question is "What is the song by George Michael?", the function matches it to the rule "new song by George Michael" and identifies the corresponding sentence "and on came george michael s new song." The matching sentences containing answers to the user's question are then printed. If no matches are found, it indicates "No matching sentences found."

In [23]:
import re
# defining the sentences taken after cosine-similarity
sentences = [
    "and i turned on the radio and on came george michael s new song .",
    "i wa a big george michael fan .",
    "there s water dripping down the wall and george michael s leather jacket is on fire and the model are naked .",
    "what i remember most is the day being very long and at the end of the day the red wine would come out .",
    "it wa his song ."
]

# Define a function to match a question to sentences based on rules
def match_question_to_sentence(question, sentences):
    rules = {
        "new song by George Michael": "and on came george michael s new song",
        "George Michael fan": "i wa a big george michael fan",
        "George Michael's leather jacket": "there s water dripping down the wall and george michael s leather jacket is on fire",
        "Long day with red wine": "what i remember most is the day being very long and at the end of the day the red wine would come out",
        "His song": "it wa his song"
    }
    matching_sentences = []
    
    # Apply rules to identify matching sentences
    for rule in rules:
        pattern = re.compile(rules[rule])
        for sentence in sentences:
            match = pattern.search(sentence)
            if match:
                matching_sentences.append((rule, sentence))
    return matching_sentences

user_question = "What is the song by George Michael?"
# taking the question given as input by the user
# Match user question to sentences
matching_sentences = match_question_to_sentence(user_question, sentences)
# Print matching sentences containing the answer
if matching_sentences:
    print("Sentences containing the answer to the user's question:")
    for rule, sentence in matching_sentences:
        print(f"Rule: {rule}, Sentence: {sentence}")
else:
    print("No matching sentences found.")

Sentences containing the answer to the user's question:
Rule: new song by George Michael, Sentence: and i turned on the radio and on came george michael s new song .
Rule: George Michael fan, Sentence: i wa a big george michael fan .
Rule: George Michael's leather jacket, Sentence: there s water dripping down the wall and george michael s leather jacket is on fire and the model are naked .
Rule: Long day with red wine, Sentence: what i remember most is the day being very long and at the end of the day the red wine would come out .
Rule: His song, Sentence: it wa his song .


In [24]:
#print(preprocessed_sentences)

In [25]:
# testing utility
# making a list of 10 questions
test_questions = [
    "Who directed the 'Freedom! '90' music video?",
    "Name a model featured in the video.",
    "What studio was the video filmed at?",
    "Which magazine cover did Tatjana Patitz shoot with George Michael?",
    "Who was the hairdresser for the video shoot?",
    "How many views had the 'Freedom! '90' video received on YouTube by the time of George Michael's passing?",
    "What did Naomi Campbell recall about the night before the video shoot?",
    "How did Guido Palau describe his approach to styling the models?",
    "What was John Pearson's initial reaction when asked to participate in the video?",
    "Name a fashion photographer"
]

# making a list of sentences that contain the answers to these questions
test_answers = [
    "directed by david fincher who wa at the start of his film career and filmed over several day at merton park studio in london the video from mr. michael s 1990 album",
    "sure other catwalk favorite starred in music video during that era and fashion photographer like steven meisel and herb ritts were recruited by star such a madonna chris isaak and janet jackson to burnish their visuals .",
    "directed by david fincher who wa at the start of his film career and filmed over several day at merton park studio in london the video from mr. michael s 1990 album",
    "i m trying to remember what magazine it wa .",
    "it run for six and a half minute and it production also gathered a team of novice professional who would go on to become major force in fashion including the stylist camilla nickerson the hairdresser guido palau the makeup artist carol brown and the model john pearson .",
    "90 had been viewed more than 37 million time on youtube by the time of mr. michael s unexpected death at 53 last sunday .",
    "we d done a british vogue cover with peter lindbergh with all of u .",
    "my part wa to make the girl the best they could look a who they were .",
    "of course i said yes .",
    "sure other catwalk favorite starred in music video during that era and fashion photographer like steven meisel and herb ritts were recruited by star such a madonna chris isaak and janet jackson to burnish their visuals ."   
]

-  The model calculates the cosine similarity between each test question and the preprocessed sentences (the article) using TF-IDF vectorization. 
-  It iterates through each question, combines it with the preprocessed sentences, vectorizes the combined documents using TF-IDF, and computes the cosine similarity between the question vector and each sentence vector. 
-  The cosine similarity scores between each question and the preprocessed sentences are printed for evaluation. This process allows for assessing how closely each question aligns with the content of the article, aiding in question-answering tasks

In [26]:
# calculating the cosine similarities between every test question 
## and the preprocessed sentences (the article)
# combine each question with the preprocessed sentences for vectorization
documents_list = [[question] + preprocessed_sentences for question in test_questions]

# vectorize the documents using TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_list = [tfidf_vectorizer.fit_transform(documents) for documents in documents_list]

# compute cosine similarity between each question and the preprocessed sentences
for i, question in enumerate(test_questions):
    cosine_similarities = cosine_similarity(tfidf_matrix_list[i][0:1], tfidf_matrix_list[i][1:])
    print(f"For question: '{question}':")
    for j, sentence in enumerate(preprocessed_sentences):
        print(f"Cosine similarity with sentence {j + 1}: {cosine_similarities[0][j]}")
    print()  # Print an empty line for separation

For question: 'Who directed the 'Freedom! '90' music video?':
Cosine similarity with sentence 1: 0.1336607759225045
Cosine similarity with sentence 2: 0.3554250712726506
Cosine similarity with sentence 3: 0.5155863250932315
Cosine similarity with sentence 4: 0.10270019827286814
Cosine similarity with sentence 5: 0.04570029967691609
Cosine similarity with sentence 6: 0.3554250712726506
Cosine similarity with sentence 7: 0.4067846891606027
Cosine similarity with sentence 8: 0.0
Cosine similarity with sentence 9: 0.23974007093215477
Cosine similarity with sentence 10: 0.0
Cosine similarity with sentence 11: 0.0
Cosine similarity with sentence 12: 0.0
Cosine similarity with sentence 13: 0.20073874827860202
Cosine similarity with sentence 14: 0.09964280134222522
Cosine similarity with sentence 15: 0.0
Cosine similarity with sentence 16: 0.3554250712726506
Cosine similarity with sentence 17: 0.09448741550249454
Cosine similarity with sentence 18: 0.07469248011475951
Cosine similarity with se

In [27]:
# # selecting the sentences for every question, where cosine similarity is greater than 0.3
# threshold = 0.3
# selected_sentences = []

# for i, question in enumerate(test_questions):
#     relevant_sentences = []
#     for j, sentence in enumerate(preprocessed_sentences):
#         if cosine_similarities[0][j] > threshold:
#             relevant_sentences.append(sentence)
#     selected_sentences.append(relevant_sentences)
#     print(f"For question: '{question}':")
#     print(f"Selected relevant sentences:")
#     for relevant_sentence in relevant_sentences:
#         print(f"  - {relevant_sentence}")
#     print() # for space between every question

### Top 5 sentences with the highest scores for every question:- 

1. For question: 'Who directed the 'Freedom! '90' music video?':
-  Sentence 3: Cosine similarity = 0.5155863250932315
-  Sentence 6: Cosine similarity = 0.3554250712726506
-  Sentence 7: Cosine similarity = 0.4067846891606027
-  Sentence 96: Cosine similarity = 0.3554250712726506
-  Sentence 97: Cosine similarity = 0.4067846891606027

2. For question: 'Name a model featured in the video.':
-  Sentence 169: Cosine similarity = 0.2701937688767456
-  Sentence 84: Cosine similarity = 0.2707275966976312
-  Sentence 129: Cosine similarity = 0.206475056073045
-  Sentence 3: Cosine similarity = 0.21049393852034268
-  Sentence 19: Cosine similarity = 0.1622427635379964

3. For question: What studio was the video filmed at?
-  Sentence 9: Cosine similarity = 0.3274496834111561
-  Sentence 3: Cosine similarity = 0.1825296345771319
-  Sentence 66: Cosine similarity = 0.18156881863924468
-  Sentence 129: Cosine similarity = 0.17904466418002563
-  Sentence 64: Cosine similarity = 0.18572029604377902

4. For question: Which magazine cover did Tatjana Patitz shoot with George Michael?
-  Sentence 68: Cosine similarity = 0.42883241445590903
-  Sentence 20: Cosine similarity = 0.2259481895143835
-  Sentence 76: Cosine similarity = 0.23677510372879723
-  Sentence 12: Cosine similarity = 0.2042206926883365
-  Sentence 7: Cosine similarity = 0.13702696261519176

5. For question: Who was the hairdresser for the video shoot?
-  Sentence 14: Cosine similarity = 0.2328036475792126
-  Sentence 49: Cosine similarity = 0.2334026048097572
-  Sentence 13: Cosine similarity = 0.21893361276942958
-  Sentence 22: Cosine similarity = 0.19938915332125606
-  Sentence 26: Cosine similarity = 0.1822298444552918

6. For question: How many views had the 'Freedom! '90' video received on YouTube by the time of      George Michael's passing?
-  Sentence 17: Cosine similarity = 0.360417534669241
-  Sentence 3: Cosine similarity = 0.30862631580515104
-  Sentence 7: Cosine similarity = 0.24349842854904777
-  Sentence 49: Cosine similarity = 0.21275492570875676
-  Sentence 130: Cosine similarity = 0.21275492570875676

7. For question: What did Naomi Campbell recall about the night before the video shoot?
-  Sentence 19: Cosine similarity = 0.3819663207528938
-  Sentence 26: Cosine similarity = 0.3460220774349842
-  Sentence 12: Cosine similarity = 0.18190188982505548
-  Sentence 114: Cosine similarity = 0.18857430966178265
-  Sentence 129: Cosine similarity = 0.16473945261202116

8. For question: How did Guido Palau describe his approach to styling the models?
-  Sentence 49: Cosine similarity = 0.3591782818896769
-  Sentence 14: Cosine similarity = 0.14270811017190568
-  Sentence 47: Cosine similarity = 0.13293461720447466
-  Sentence 44: Cosine similarity = 0.11579415511226474
-  Sentence 64: Cosine similarity = 0.11545631874040485

9. For question: What was John Pearson's initial reaction when asked to participate in the video?
-  Sentence 102: Cosine similarity = 0.3747287253476217
-  Sentence 129: Cosine similarity = 0.295285396024703
-  Sentence 43: Cosine similarity = 0.15143179908786764
-  Sentence 14: Cosine similarity = 0.14626829228220783
-  Sentence 47: Cosine similarity = 0.13657427628198401

10. For question: Name a fashion photographer
-  Sentence 13: Cosine similarity = 0.20381843214357342
-  Sentence 4: Cosine similarity = 0.18319426961231916
-  Sentence 84: Cosine similarity = 0.1515765460438333
-  Sentence 1: Cosine similarity = 0.14693191011969936
-  Sentence 173: Cosine similarity = 0.19694275109618717

## H. Calculating MRR

MRR is a metric used to evaluate systems that return a ranked list of results. It measures the average of the reciprocal ranks of the first relevant document retrieved for each query. MRR is particularly useful when only the top-ranked result is considered relevant, as is often the case in question-answering tasks (Evidently AI n.d.). I chose MRR over the other metrics (F1 score and MAP) due to its simplicity, alignment with task goals, and focus on the top-ranked result, making it a suitable choice for evaluating the effectiveness of such systems. MRR is a straightforward metric that captures the effectiveness of the system in returning relevant results without considering the entire ranked list. This simplicity makes it easy to understand and interpret.

In [28]:
# calculating MRR for the test questions
def calculate_mrr(questions, answers):
    total_mrr = 0
    num_questions = len(questions)

    for i in range(num_questions):
        correct_answer = answers[i]
        rank = 0

        # check if the correct answer exists in the provided answers
        if correct_answer in answers:
            rank = answers.index(correct_answer) + 1  
            # index starts from 0, so add 1 to get rank
        # calculate MRR for the current question
        if rank > 0:
            mrr = 1 / rank
            total_mrr += mrr
        else:
            print(f"Correct answer not found for question {i + 1}")
    # calculate overall MRR
    overall_mrr = total_mrr / num_questions if num_questions > 0 else 0
    return overall_mrr

# calculate MRR
mrr = calculate_mrr(test_questions, test_answers)
print("Mean Reciprocal Rank (MRR):", mrr)

Mean Reciprocal Rank (MRR): 0.39956349206349207


In [29]:
#!pip show huggingface-hub

## I. Conclusion

In conclusion, this project focused on developing and evaluating a question-answering system using natural language processing techniques. The process involved several key steps namely preprocessing the text data, vectorization of text data, evaluation of the question-answering system and testing the system. Overall, the project demonstrated the effectiveness of natural language processing techniques in building question-answering systems and highlighted the importance of choosing appropriate evaluation metrics based on the task requirements. Moving forward, further refinements and optimizations could be explored to enhance the system's performance and usability in real-world applications. The answering to the question can be made more accurate by training the model with a fine-tuned BERT. The system’s accuracy in extracting the exact, correct answer can be increased by making the pre-processing of the text better. The scope of this project is limited to a single, one-time question-answer. 

## J. References 

[1] Zheng J, Chapman W W, Crowley R S & Savova G K 2011, ‘Coreference resolution: A review of general methodologies and applications in the clinical domain’, Journal of Biomedical Informatics, vol. 44, issue. 6, pp. 1113–1122. 

[2] Jurafsky D & Martin J H 2023, Speech and Language Processing, <https://web.stanford.edu/~jurafsky/slp3/ed3book_jan72023.pdf>

[3] Institute of Electrical and Electronics Engineers. (n.d.), ICUMT 2019, the 11th International       Congress on Ultra Modern Telecommunications and Control Systems.

[4] Pratik Nabriya 2024, What is Exploratory Data Analysis (EDA) and how does it work?, Analytics Vidhya, <https://www.analyticsvidhya.com/blog/2021/08/exploratory-data-analysis-and-visualization-techniques-in-data-science/>

[5] Sunny Srinidhi 2013, Lemmatization in Natural Language Processing (NLP) and Machine Learning, Built In,  <https://builtin.com/machine-learning/lemmatization>

[6] Adwaith Dhawan R 2024, A beginner’s introduction to Named Entity Recognition (NER), Anlaytics Vidhya, <https://www.analyticsvidhya.com/blog/2021/11/a-beginners-introduction-to-ner-named-entity-recognition/>

[7] Explosion.ai 2022, End-to-end Neural Coreference Resolution in spaCy, Explosion.ai, <https://explosion.ai/blog/coref>

[8] The Stanford NLP Group n.d., Coreference Resolution, The Stanford NLP Group,  <https://nlp.stanford.edu/projects/coref.shtml#:~:text=Coreference%20resolution%20is%20the%20task,question%20answering%2C%20and%20information%20extraction.>

[9] coreference-resolution n.d., <https://github.com/Python-Tutorials-for-Digital-Humanities/coreference-resolution/blob/main/prepared.ipynb>

[10] Evidently AI n.d., Mean Reciprocal Rank (MRR) explained, Evidently AI, <https://www.evidentlyai.com/ranking-metrics/mean-reciprocal-rank-mrr#:~:text=Mean%20Reciprocal%20Rank%20(MRR)%20is%20a%20ranking%20quality%20metric.,of%20the%20first%20relevant%20item.>
